In [1]:
#Importing Dash and pandas for Dashboard
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import pandas as pd
import pandas_datareader.data as web
import datetime

In [26]:
# Building Sample dashboard layout using Dash app on Python 
# Importing data using padas built in data reader and Stooq data (this will be eventually be modified to have user query)
start = datetime.datetime(2021, 1, 1)
end = datetime.datetime(2021, 6, 30)
df = web.DataReader(['AAPL','GME','AMC','CLNE','CLOV','TSLA'],
                    'stooq', start=start, end=end)

df = df.stack().reset_index()
df.rename(columns={'level_0': 'Date'})
print(df[:15])
#df.head(15)

Attributes       Date Symbols   Close      High       Low      Open    Volume
0          2021-06-30    AAPL  136.96  137.4100  135.8700  136.1700  63261393
1          2021-06-30     GME  214.14  215.2787  206.9379  209.6900   2539648
2          2021-06-30     AMC   56.68   58.1800   54.6500   56.0000  59020626
3          2021-06-30    CLNE   10.15   10.3499    9.9500   10.2901  12858058
4          2021-06-30    CLOV   13.32   14.1700   13.1700   13.6000  34028169
5          2021-06-30    TSLA  679.70  692.8100  678.1400  679.7700  18924862
6          2021-06-29    AAPL  136.33  136.4900  134.3500  134.8000  64556081
7          2021-06-29     GME  210.88  215.2200  208.0100  213.5900   2480009
8          2021-06-29     AMC   56.43   61.0000   56.1800   59.0400  63604135
9          2021-06-29    CLNE   10.31   11.1000   10.2330   10.8200  16425943
10         2021-06-29    CLOV   13.36   14.4400   12.5400   12.7100  73090601
11         2021-06-29    TSLA  680.76  687.5099  675.8900  684.6

In [48]:
# Calls Dash app using Jupyter notebook and sets the themme using bootstrap dark
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.DARKLY],
                  meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])


# Defines the overall layout of the dashboard using dash and htmlcode. 
# Dash components contain graphs and menus from plotly 
app.layout = html.Div([
    #Set Title of the dashboard
    html.H1("WallStreetBets Dashboard",className='text-center text-primary mb-4'),
    
    #Added drop down menus for closing price graph and volume graphs
    html.Div([
        html.Div([
            dcc.Dropdown(id='my-dpdn', multi=False, value='GME',style={'color': 'black'},
                         options=[{'label':x, 'value':x}
                                  for x in sorted(df['Symbols'].unique())],
                         ),
            dcc.Graph(id='line-fig', figure={})]),
        
        html.Div([
            dcc.Dropdown(id='my-dpdn2', multi=True, value=['GME','TSLA'],style={'color': 'black'},
                         options=[{'label':x, 'value':x}
                                  for x in sorted(df['Symbols'].unique())],
                         ),
            dcc.Graph(id='line-fig2', figure={})])          
    ],),
      
           
    #Embedded tableau dashboard for machine learning components 
    html.Iframe(src="https://public.tableau.com/app/profile/lisette.worster/viz/SentimentAnalysis_16252430553980/Storyboard",
                style={"height": "1067px", "width": "100%"})
])



# Callback section: connecting the components
# ************************************************************************
# Line chart - Single
@app.callback(
    Output('line-fig', 'figure'),
    Input('my-dpdn', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols']==stock_slctd]
    figln = px.line(dff, x='Date', y='Close',
                    
                    
                    
                    template='plotly_dark').update_layout(
                                            {'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                                                'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    return figln


# Line chart - multiple
@app.callback(
    Output('line-fig2', 'figure'),
    Input('my-dpdn2', 'value')
)
def update_graph(stock_slctd):
    dff = df[df['Symbols'].isin(stock_slctd)]
    figln2 = px.line(dff, x='Date', y='Volume', color='Symbols',
                     
                     
                     
                     template='plotly_dark').update_layout(
                                            {'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                                                'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    return figln2
#Runs dashboard through Jupyter notebook and opens it in another window
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


In [46]:
#Terminates activity on port
app._terminate_server_for_port("localhost", 8050)

In [ ]:
# Running another form of ML Section 

In [47]:
wallbets_df = pd.read_csv('reddit_wsb.csv')
wallbets_df.head()
wallbets_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50070 entries, 0 to 50069
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      50070 non-null  object 
 1   score      50070 non-null  int64  
 2   id         50070 non-null  object 
 3   url        50070 non-null  object 
 4   comms_num  50070 non-null  int64  
 5   created    50070 non-null  float64
 6   body       23295 non-null  object 
 7   timestamp  50070 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.1+ MB
